In [2]:
import gradio as gr
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler
import requests
from PIL import Image
from io import BytesIO

from scripts.img2img import StableDiffusionImg2ImgPipeline, preprocess

In [3]:
lms = LMSDiscreteScheduler(
    beta_start=0.00085, 
    beta_end=0.012, 
    beta_schedule="scaled_linear"
)

pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", 
    scheduler=lms,
    #revision="fp16", 
    use_auth_token=True
).to("cuda")

pipeimg = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    #revision="fp16", 
    #torch_dtype=torch.float16,
    use_auth_token=True
).to("cuda")

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.70 GiB total capacity; 415.36 MiB already allocated; 12.00 MiB free; 448.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
base_prompt = ", high-quality, Pixiv, WLOP, Greg Rutkowski, ArtStation. Oil on canvas."

block = gr.Blocks(css=".container { max-width: 800px; margin: auto; }")

num_samples = 1

def infer(prompt, init_image, strength):
    if init_image != None:
        init_image = init_image.convert("RGB").resize((512, 512))
        #init_image = init_image.resize((512, 512))
        init_image = preprocess(init_image)
        with autocast("cuda"):
            images = pipeimg([prompt] * num_samples, init_image=init_image, strength=strength, guidance_scale=7.5, num_inference_steps=150)["sample"]
    else: 
        with autocast("cuda"):
            images = pipe([prompt] * num_samples, guidance_scale=7.5)["sample"]

    return images


with block as demo:
    gr.Markdown("<h1><center>Stable Diffusion</center></h1>")
    gr.Markdown("Provide a text prompt and (optionally) an image")
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=True):

                text = gr.Textbox(
                    label="Enter your prompt", show_label=False, max_lines=1
                ).style(
                    border=(True, False, True, True),
                    rounded=(True, False, False, True),
                    container=False,
                )
                btn = gr.Button("Run").style(
                    margin=False,
                    rounded=(False, True, True, False),
                )
        strength_slider = gr.Slider(
            label="Strength",
            maximum = 1,
            value = 0.75         
        )
        image = gr.Image(
            source="canvas",
            label="Intial Image",
            type="pil"
        )
               
        gallery = gr.Gallery(label="Generated images", show_label=False).style(
            grid=[2], height="auto"
        )
        text.submit(infer, inputs=[text,image,strength_slider], outputs=gallery)
        btn.click(infer, inputs=[text,image,strength_slider], outputs=gallery)

demo.launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7860/


KeyboardInterrupt: 

In [23]:
#base_prompt = " with vibrant color background, photorealistic, dynamic lighting, beautiful, trending on artstation, wallpaper, dream, 4 k, award winning, lovely pastel colors, ethereal, elegant "
base_prompt = ", high-quality, Pixiv, WLOP, Greg Rutkowski, ArtStation. Oil on canvas."
def predict(image, prompt):
  prompt += base_prompt
  init_img =  image.convert("RGB").resize((512, 512)) 
  init_img = preprocess(init_img)
  with autocast("cuda"):
    images = pipeimg(prompt=prompt, init_image=init_img, strength=0.8)["sample"]

  return(images[0])

In [ ]:
gr.Interface(
    predict,
    title = 'img2img',
    inputs=[
        gr.Image(source = 'canvas', type = 'pil'),
        gr.Textbox(label = 'prompt')
    ],
    outputs = [
        gr.Image()
        ]
).launch(debug=True, share=True)

Running on local URL:  http://127.0.0.1:7862/
Running on public URL: https://22309.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


/tmp/ipykernel_727636/3795008547.py:8: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((w, h), resample=PIL.Image.LANCZOS)


0it [00:00, ?it/s]

/tmp/ipykernel_727636/3795008547.py:8: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((w, h), resample=PIL.Image.LANCZOS)


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

/tmp/ipykernel_727636/3795008547.py:8: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((w, h), resample=PIL.Image.LANCZOS)


0it [00:00, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
/tmp/ipykernel_727636/3795008547.py:8: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize((w, h), resample=PIL.Image.LANCZOS)


0it [00:00, ?it/s]